In [5]:
import h5py
import mne
from eegatscale.models import LinearHeadBENDR
from scripts.finetune_cv import H5PYDatasetLabeled
from scripts.finetune_cv_nogroups import H5PYDatasetLabeled as H5PYDatasetLabeledNoGroups
from torch.utils.data import DataLoader
from eegatscale.transforms import StandardizeLabel, Standardize
import numpy as np
import torch
import pickle
import os
# from dataset_utils import PickleDataset
from tqdm import tqdm
from glob import glob
import warnings
warnings.filterwarnings("ignore")

# TUSZ data

In [6]:
number = 2
# tusz_model_dir = f"/scratch/s194101/finetune_logs_all/tusz_noica_noica_60.0_nogroups_{number}/"
# tusz_model_dir = f"/scratch/s194101/finetune_logs_all/tusz_noica_noica_60.0_nogroups_whole_dataset/"

# whole dataset split train
tusz_model_dir = "/scratch/s194101/finetune_logs_all/tusz_noica_noica_60.0_nogroups_whole_dataset_split_train"

models = glob(f"{tusz_model_dir}/*/*/*.ckpt")

# we need to test all the models on the eval set
data_dir_eval = "/scratch/s194101/data/preprocessed_downstream/tusz/noica_60.0_titans_combine_eval_combined"
data_dir_train = "/scratch/s194101/data/preprocessed_downstream/tusz/noica_60.0_titans_combine_whole_dataset_combined"
data_dir_split_train = "/scratch/s194101/data/preprocessed_downstream/tusz/noica_60.0_titans_combine_whole_dataset_split_train_combined"
data_dir_split_eval = "/scratch/s194101/data/preprocessed_downstream/tusz/noica_60.0_titans_combine_whole_dataset_split_eval_combined"
transform = StandardizeLabel()


# data_eval = H5PYDatasetLabeledNoGroups(data_dir_eval, transform=transform)
# data_eval = H5PYDatasetLabeledNoGroups(data_dir_train, transform=transform)
# data_eval = H5PYDatasetLabeledNoGroups(data_dir_split_train, transform=transform)
data_eval = H5PYDatasetLabeledNoGroups(data_dir_split_eval, transform=transform)
print(f"{len(data_eval)} samples in eval set")



dataloader_eval = DataLoader(data_eval, batch_size=80, shuffle=True)

912 samples in eval set


In [7]:
# x, y = next(iter(dataloader_eval))
# print(x.shape, y.shape)

# y_pred = model(x)

# y_pred.argmax(dim=1), y

# # y_pred.argmax(dim=1) == y

In [ ]:
best_acc = 0
best_sem = 0
best_model = None

for model_path in tqdm(models):
    print(f"Testing model {model_path}")
    model = LinearHeadBENDR(model_path, encoder_h=512, in_features=19, out_features=2)
    state_dict = torch.load(model_path, map_location=torch.device('cpu'))["state_dict"]
    model.load_state_dict(state_dict)
    model.eval()
    print("model loaded")


    # test model on eval set
    accs = []
    with torch.no_grad():
        for i, batch in enumerate(dataloader_eval):
            x, y = batch
            y_pred = model(x)
            batch_accuracy = (y_pred.argmax(dim=1) == y).float().mean().item()
            accs.append(batch_accuracy)
            # print(f"Batch {i}, accuracy: {batch_accuracy}")
        
        # standard error of the mean
        accs = np.array(accs)
        sem = np.std(accs) / np.sqrt(len(accs))


        print(f"Model {model_path} accuracy on eval set: {np.mean(accs)} +- {sem}")
        if np.mean(accs) > best_acc:
            best_acc = np.mean(accs)
            best_sem = sem
            best_model = model_path
        
        print(f"Best model so far: accuracy: {best_acc} +- {best_sem} | {best_model}")


  0%|          | 0/50 [00:00<?, ?it/s]

Testing model /scratch/s194101/finetune_logs_all/tusz_noica_noica_60.0_nogroups_whole_dataset_split_train/version_36/checkpoints/epoch=3-step=148.ckpt
Initialized mask embedding and position encoder from  /scratch/s194101/finetune_logs_all/tusz_noica_noica_60.0_nogroups_whole_dataset_split_train/version_36/checkpoints/epoch=3-step=148.ckpt
model loaded


### on eval set:

Best model so far: accuracy: 0.6795454545454546 +- 0.013922615366108731 | /scratch/s194101/finetune_logs_all/tusz_noica_noica_60.0_nogroups_4/version_9/checkpoints/epoch=14-step=150.ckpt

Best model so far: accuracy: 0.768560604615645 +- 0.011691771714137506 | /scratch/s194101/finetune_logs_all/tusz_noica_noica_60.0_nogroups_whole_dataset/version_47/checkpoints/epoch=2-step=141.ckpt

### on test set:

Best model so far: accuracy: 0.979166673289405 +- 0.002758420816857501 | /scratch/s194101/finetune_logs_all/tusz_noica_noica_60.0_nogroups_whole_dataset/version_1/checkpoints/epoch=13-step=658.ckpt

Best model so far: accuracy: 0.9757505257924398 +- 0.002019294366371989 | /scratch/s194101/finetune_logs_all/tusz_noica_noica_60.0_nogroups_whole_dataset_split_train/version_54/checkpoints/epoch=13-step=518.ckpt

# MMIDB data